### Data Center Siting Relative to Electricity Infrastructure and Water Resources in Texas Using Geospatial Methods

Shalini Das
<br> GGIS 403 (Fall 2025)

### Data Sources </u> 
* Data Center Locations and Attributes: Retrieved 382 of 392 data center locationfrom datacentermaps.com
* Electricity Generating Power Plant Locations: Retrieved latitude and longitude coordinates using 'plantid' and 'plantName' as index to estimate geographic impact of  Fuel Mix (EIA 930) data within specific counties/regions and how this may correspond with data center siting once aggregated/preprocessed.
* Fuel Energy Mix Data: Sourced from Form EIA-923 Monthly Generation by Energy Source, filtered by Balancing Authority/Form Respondent and selected: ERCOT (Electric Reliability Council of Texas), EPE (El Paso Electric Company), SWPP (Southwest Power Pool for Southeastern Texas bordering Louisiana), and (TEX). 
* Weather Data: Retrieved hourly temperature, humidity, and precipitation, wet bulb temperature at coordinates retrieved for datacenter locations from July 1, 2022 through July 31, 2025 (37 month period). 
* ERCOT Zone Data: Incorporated in order to join and standardize data from disparate sources across a common geographic unit (counties), and then perform spatial operations in order to perform analysis across distinct regulatory/regional entities in Texas.

### Setup Requirements
1. **EIA Data**: https://api.eia.gov/v2/electricity/ (requires free API key at https://www.eia.gov/opendata/register.php)
2. **Weather Data**: https://archive-api.open-meteo.com/v1/archive (free, no key required)
3. **API Rate Limits**: Approximately 60-second delays between API batch requests for EIA and Open-Meteo
4. **Data Center Locations Scraper**: expected runtime for full data collection and saving to .csv files locally is approximately 3 hours

### External Data Sources
```
Texas Datacenter Siting Project
├── texas_datacenter_scraper.py
├── scraper_debug.py
├── https://www.datacentermap.com/usa/texas/
├── ERCOT Wind Regions to County Mapping.xlsx: https://www.ercot.com/files/docs/2024/05/31/Wind%20and%20Solar%20Regions%20to%20County%20Mapping.xlsx
├── ERCOT Solar Regions to County Mapping.xlsx: https://www.ercot.com/files/docs/2024/05/31/Wind%20and%20Solar%20Regions%20to%20County%20Mapping.xlsx
├── hourly_weather_data.csv: https://archive-api.open-meteo.com/v1/archive
├── geojson-counties-fips.json
└── 
```

In [1]:
#Import Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import contextily as ctx
import geopandas as gpd
import libpysal as ps 
from esda import Moran, Moran_Local
from splot.esda import moran_scatterplot, lisa_cluster
from shapely.geometry import Point, polygon, box
import rasterio
import raster4ml
import requests
import openmeteo_requests
import requests_cache
from retry_requests import retry
from dotenv import load_dotenv
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from mgwr.gwr import GWR,MGWR
from mgwr.sel_bw import Sel_BW

### A) Data Center Coordinates

In [2]:
# Load Datacenters csv from scraped and cleaned data using 'scrape_datacenters.py' and 'clean_datacenters.py'
datacenters = pd.read_csv("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/processed/texas_datacenters_final_clean.csv")
datacenters['whitespace_ratio'] = datacenters['whitespace_sqft'] / datacenters['building_size_sqft']
datacenters = datacenters.dropna(subset=['latitude', 'longitude'])
# Convert to datacenters.csv to GeoDataFrame
geometry = [Point(xy) for xy in zip(datacenters['longitude'], datacenters['latitude'])]
tx_datacenters_gdf = gpd.GeoDataFrame(datacenters, geometry=geometry, crs='EPSG:4326') #EPSG is the World Geodetic System 1984 (WGS 84) per EP

In [3]:
#Load ERCOT Wind and Solar Zones to County Mapping
ercot_wind_zones_to_county = pd.read_excel("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/external/ERCOT Wind Regions to County Mapping.xlsx")
ercot_solar_zones_to_county = pd.read_excel("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/external/ERCOT Solar Regions to County Mapping.xlsx")
# Merge Wind and Solar Zones to County Mapping
counties = pd.merge(ercot_wind_zones_to_county, ercot_solar_zones_to_county, on='County', how='outer')
counties = counties.drop_duplicates(subset=['County'])
counties = counties[['County', 'Solar Region', 'Wind Region']]
counties = counties.rename(columns={'County': 'county'})

# Load County shapefile
tx_counties = gpd.read_file('/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/external/geojson-counties-fips.json')
tx_counties = tx_counties[tx_counties['STATE'] == '48'] #Filter to Texas only)

# Add County Polygons to counties df for spatial analysis
counties_gdf = tx_counties.merge(counties, left_on='NAME', right_on='county', how='left')
counties_gdf = counties_gdf[['county', 'Solar Region', 'Wind Region', 'geometry']]

# Spatial Join Datacenters with Counties
datacenters_counties = gpd.sjoin(tx_datacenters_gdf, tx_counties, how='left', predicate='within')
datacenters_counties = datacenters_counties.rename(columns={'NAME': 'county', 'name':'datacenter'})
datacenters_counties = datacenters_counties.merge(counties, on='county', how='left')
datacenters_counties = datacenters_counties[['datacenter', 'operator', 'address', 'city', 'postal_code', 'county', 'Solar Region', 'Wind Region', 'power_capacity_mw', 'building_size_sqft', 'whitespace_sqft', 'whitespace_ratio', 'geometry', 'latitude', 'longitude']]
datacenters_counties

,datacenter,operator,address,city,postal_code,county,Solar Region,Wind Region,power_capacity_mw,building_size_sqft,whitespace_sqft,whitespace_ratio,geometry,latitude,longitude
0,Datacenter Park - Dallas,Follow on LinkedIn,900-908 Quality Way,Richardson,75081.0,Dallas,CenterEast,North,NaN,NaN,NaN,NaN,POINT (-96.71217 32.96608),32.966085,-96.712168
1,Digital Realty Garland Campus,Follow on LinkedIn,1702 W Campbell Rd / 1502 Ferris Rd / 1505 Fer...,Garland,75044.0,Dallas,CenterEast,North,NaN,NaN,NaN,NaN,POINT (-96.65048 32.9771),32.977103,-96.650478
2,908 Quality Way (DFW20),Follow on LinkedIn,908 Quality Way,Richardson,75081.0,Dallas,CenterEast,North,10.0,NaN,38000.0,NaN,POINT (-96.70999 32.96622),32.966215,-96.709990
3,904 Quality Way (DFW21),Follow on LinkedIn,904 Quality Way,Richardson,75081.0,Dallas,CenterEast,North,NaN,NaN,62600.0,NaN,POINT (-96.71115 32.96625),32.966249,-96.711150
4,1702 W Campbell Rd,Follow on LinkedIn,1702 W Campbell Rd,Garland,75044.0,Dallas,CenterEast,North,NaN,NaN,NaN,NaN,POINT (-96.65048 32.9771),32.977103,-96.650478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,Lumen El Paso 2,Lumen,201 E Main Drive,El Paso,79901,NaN,NaN,NaN,NaN,3810.0,NaN,NaN,POINT (-106.48845 31.7603),31.760298,-106.488455
378,Lumen El Paso 1,Lumen,501 W. Overland,El Paso,79936,NaN,NaN,NaN,NaN,20000.0,NaN,NaN,POINT (-106.49339 31.75585),31.755853,-106.493388
379,Project Dorothy I,Soluna Computing,9985 Co Rd F,Silverton,79257,Briscoe,NorthWest,Panhandle,286.0,NaN,NaN,NaN,POINT (-101.31759 34.39229),34.392291,-101.317591
380,Duos Edge AI - Pampa,"Duos Edge AI, Inc.","Pampa Energy Center, 8201 Fm 2300",Pampa,79065,Gray,NorthWest,Panhandle,50.0,NaN,NaN,NaN,POINT (-101.04402 35.48546),35.485458,-101.044017


In [4]:
def classify_datacenters_by_power_size(power_capacity_mw):
    if power_capacity_mw is None or pd.isna(power_capacity_mw):
        return 'Missing Data'

    # Check if power_capacity_mw is within reasonable bounds
    if power_capacity_mw < 0.075:  # Minimum realistic datacenter (your data shows 0.075 MW)
        return "Unrealistic (Too Small)"
    elif power_capacity_mw > 800:  # Maximum in your dataset
        return "Unrealistic (Too Large)"
        
    # Classify based on capacity size
    if power_capacity_mw < 5:
        return 'Small/Edge (<5 MW)'
    elif power_capacity_mw < 50:
        return 'Medium (5-50 MW)'
    elif power_capacity_mw < 200:
        return 'Large (50-200 MW)'
    else:
        return 'Hyperscale (200+ MW)'

datacenters['power_size_category'] = datacenters['power_capacity_mw'].apply(classify_datacenters_by_power_size)
datacenters_counties['power_size_category'] = datacenters_counties['power_capacity_mw'].apply(classify_datacenters_by_power_size)

### B) Weather Based WUE Data

Pull data for 382 data center coordinates using Open-Meteo API. 

In [5]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


def fetch_weather_data(gdf, start_index=0, end_index=None, batch_size=50):
    if end_index is None:
        latitudes = gdf['latitude'].tolist()[start_index:]
        longitudes = gdf['longitude'].tolist()[start_index:]
        datacenter_names = gdf['datacenter'].tolist()[start_index:]
        end_index = len(gdf)
    else:
        latitudes = gdf['latitude'].tolist()[start_index:end_index]
        longitudes = gdf['longitude'].tolist()[start_index:end_index]
        datacenter_names = gdf['datacenter'].tolist()[start_index:end_index]
    
    total_datacenters = len(latitudes)
    successful = 0
    failed = 0
    combined_hourly_dc_weather = []
    
    print(f"Fetching weather data for {len(latitudes)} datacenters...")

    for batch_start in range(0, total_datacenters, batch_size):
        batch_end = min(batch_start + batch_size, total_datacenters)
        
        try:
            print(f"\nProcessing batch {batch_start//batch_size + 1}/{(len(latitudes)-1)//batch_size + 1}")
            print(f"  Datacenters {start_index + batch_start + 1} to {start_index + batch_end}")
            
            # Get batch coordinates
            batch_lats = latitudes[batch_start:batch_end]
            batch_lons = longitudes[batch_start:batch_end]
            batch_names = datacenter_names[batch_start:batch_end]
            
            params = {
                "latitude": batch_lats,
                "longitude": batch_lons,
                "start_date": "2022-07-01",
                "end_date": "2025-07-31",
                "hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "wet_bulb_temperature_2m"],
                "temperature_unit": "fahrenheit",
                "wind_speed_unit": "mph",
                "precipitation_unit": "inch",
                "timezone": "America/Chicago",
                "cell_selection": "nearest"
            }
        
            responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
            
            # Process each location in this batch
            for i, response in enumerate(responses):
                datacenter_name = batch_names[i]
                
                # Process hourly data
                hourly = response.Hourly()
                hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
                hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
                hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
                hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
                hourly_wet_bulb_temperature_2m = hourly.Variables(4).ValuesAsNumpy()
                
                # Create datetime index
                hourly_data = {
                    "date": pd.date_range(
                        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                        freq=pd.Timedelta(seconds=hourly.Interval()),
                        inclusive="left")
                }
                
                hourly_data["datacenter"] = datacenter_name
                hourly_data["latitude"] = response.Latitude()
                hourly_data["longitude"] = response.Longitude()
                hourly_data["elevation_m"] = response.Elevation()
                hourly_data["temperature_2m"] = hourly_temperature_2m
                hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
                hourly_data["dew_point_2m"] = hourly_dew_point_2m
                hourly_data["precipitation"] = hourly_precipitation
                hourly_data["wet_bulb_temperature_2m"] = hourly_wet_bulb_temperature_2m
                
                hourly_dataframe = pd.DataFrame(data=hourly_data)
                combined_hourly_dc_weather.append(hourly_dataframe)
                successful += 1
            
            print(f"  ✓ Batch complete: {len(responses)} datacenters processed")

            if batch_end < len(latitudes):
                print(f" Wait 60 seconds before next batch: ")
                time.sleep(60)
                
        except Exception as e:
            print(f"  X Batch failed: {str(e)[:100]}")
            failed += (batch_end - batch_start)
            if "Minute" in str(e) or "rate" in str(e).lower():
                print(f"  Wait 120 seconds (rate limited): ")
                time.sleep(120)
            continue

    # Combine all dataframes
    print("\n" + "="*50)
    print("Combining hourly weather data for all processed datacenters...")
    weather_df = pd.concat(combined_hourly_dc_weather, ignore_index=True)

    print(f"\n✓ Complete!")
    print(f"  Successful: {successful}")
    print(f"  Failed: {failed}")
    print(f"  Total rows: {len(weather_df):,}")
    print(f"  Date range: {weather_df['date'].min()} to {weather_df['date'].max()}")
    print(f"  Datacenters: {weather_df['datacenter'].nunique()}")
    print(f"  Hours per datacenter: {len(weather_df) // weather_df['datacenter'].nunique():,}")

    return weather_df

In [6]:
# weather_df_batch0 = fetch_weather_data(datacenters_counties, start_index=0)

In [7]:
# weather_df_batch0.to_csv("/mnt/c/Users/shali/OneDrive/Documents/GGIS 403 (Geographic Information Science and Systems)/Final Project/hourly_weather_data.csv", index=False)

In [8]:
# weather_df_batch1 = fetch_weather_data(datacenters_counties, start_index=300)

In [9]:
# weather_df = pd.concat([weather_df_batch0, weather_df_batch1], ignore_index=True)
# weather_df.to_csv("/mnt/c/Users/shali/OneDrive/Documents/GGIS 403 (Geographic Information Science and Systems)/Final Project/hourly_weather_data.csv", index=False)
# geometry = [Point(lon, lat) for lon, lat in zip(weather_df['longitude'], weather_df['latitude'])]
# weather_gdf = gpd.GeoDataFrame(weather_df, geometry=geometry, crs="EPSG:4326")
# weather_gdf.head()

In [10]:
weather_df = pd.read_csv("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/processed/hourly_weather_data.csv")
geometry = [Point(lon, lat) for lon, lat in zip(weather_df['longitude'], weather_df['latitude'])]
weather_gdf = gpd.GeoDataFrame(weather_df, geometry=geometry, crs="EPSG:4326")
weather_gdf.head()

,date,datacenter,latitude,longitude,elevation_m,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,wet_bulb_temperature_2m,geometry
0,2022-07-01 05:00:00+00:00,Datacenter Park - Dallas,32.934971,-96.727936,198.0,81.762800,69.006970,70.6028,0.0,73.858080,POINT (-96.72794 32.93497)
1,2022-07-01 06:00:00+00:00,Datacenter Park - Dallas,32.934971,-96.727936,198.0,80.952800,71.066826,70.6928,0.0,73.653595,POINT (-96.72794 32.93497)
2,2022-07-01 07:00:00+00:00,Datacenter Park - Dallas,32.934971,-96.727936,198.0,80.772804,69.967830,70.0628,0.0,73.194275,POINT (-96.72794 32.93497)
3,2022-07-01 08:00:00+00:00,Datacenter Park - Dallas,32.934971,-96.727936,198.0,79.512800,69.192770,68.5328,0.0,71.817700,POINT (-96.72794 32.93497)
4,2022-07-01 09:00:00+00:00,Datacenter Park - Dallas,32.934971,-96.727936,198.0,78.162796,72.332560,68.5328,0.0,71.369576,POINT (-96.72794 32.93497)


Aggregate Hourly Weather Data for Each Data Center

In [11]:
datacenter_agg_weather = weather_gdf.groupby('datacenter').agg({
    'wet_bulb_temperature_2m': ['mean', 'max', 'std'],
    'temperature_2m': ['mean', 'max'],
    'precipitation': ['mean','sum'],
    'relative_humidity_2m': 'mean',
    'geometry': 'first'}).reset_index()
datacenter_agg_weather.columns = ['_'.join(col).strip('_') for col in datacenter_agg_weather.columns]
datacenter_agg_weather = datacenter_agg_weather.rename(columns={'geometry_first': 'geometry'})

In [12]:
datacenter_agg_weather = datacenters_counties.merge(datacenter_agg_weather, on=('datacenter'), how='inner')

### C) Fuel Mix Mapped by County to ERCOT Zone/Process ERCOT fuel mix data by zone, county, region

In [13]:
load_dotenv()
api_key = os.getenv("EIA_API_KEY")
print(f"Key loaded: {api_key is not None}")

Key loaded: True


In [14]:
def fetch_eia_plant_inventory(offset=0, length=5000):
    url = "https://api.eia.gov/v2/electricity/operating-generator-capacity/data/"
    params = {
        "api_key" : api_key,
        "frequency": "monthly",
        "data[0]": "county",
        "data[1]": "latitude",
        "data[2]": "longitude",
        "data[3]": "nameplate-capacity-mw",
        "data[4]": "net-summer-capacity-mw",
        "data[5]": "net-winter-capacity-mw",
        "data[6]": "operating-year-month",
        "facets[stateid][]" : "TX",
        "start": "2022-07",
        "end": "2025-07",
        "sort[0][column]": "plantid",
        "sort[0][direction]": "asc",
        "sort[1][column]": "period",
        "sort[1][direction]": "asc", 
        "offset": offset,
        "length": length
    }

    response = requests.get(url, params=params)
    response.raise_for_status()
    
    eia_plant_inventory = response.json()
    return eia_plant_inventory
    
def fetch_facility_fuel(offset: int = 0, length: int = 5000): 
    url = "https://api.eia.gov/v2/electricity/facility-fuel/data"
    
    params = {
        "api_key" : api_key,
        "frequency": "monthly",
        "data[0]": "average-heat-content",
        "data[1]": "consumption-for-eg-btu",
        "data[2]": "generation",
        "facets[state][]": "TX",
        "start": "2022-07",
        "end": "2025-07",
        "sort[0][column]": "plantCode",
        "sort[0][direction]": "asc",
        "sort[1][column]": "period",
        "sort[1][direction]": "asc", 
        "offset": offset,
        "length": length
    }
        
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    plant_fuel_mix = response.json()
    return plant_fuel_mix 

In [15]:
def fetch_all_eia_pages(fetch_function, max_retries=5, sleep_time=1):
    """
    Fetch all pages of data by handling pagination with retry logic
    Works with functions that only take offset and length parameters
    """
    all_data = []
    offset = 0
    length = 5000
    
    print(f"Starting pagination for {fetch_function.__name__}...")
    
    while True:
        retry_count = 0
        success = False
        result = None
        
        while retry_count < max_retries and not success:
            try:
                # Call the function with only offset and length
                result = fetch_function(offset=offset, length=length)
                success = True
                
            except requests.exceptions.RequestException as e:
                retry_count += 1
                if retry_count < max_retries:
                    wait_time = sleep_time * (2 ** retry_count)
                    print(f"  X Request failed (attempt {retry_count}/{max_retries}). Retrying in {wait_time}s...")
                    time.sleep(wait_time)
                else:
                    print(f"  X Failed after {max_retries} attempts: {str(e)[:100]}")
                    raise
        
        # Handle the JSON response from your functions
        if result and 'response' in result and 'data' in result['response']:
            df = pd.DataFrame(result['response']['data'])
            print(f"  ✓ Fetched {len(df):,} records (total so far: {sum(len(d) for d in all_data) + len(df):,})")
            
            if df.empty or len(df) == 0:
                print("No more data to fetch.")
                break
                
            all_data.append(df)
            
            # If we got fewer records than requested, we're done
            if len(df) < length:
                print(f"Received {len(df)} records (less than {length}). Pagination complete.")
                break
        else:
            print("No data found in response.")
            break
            
        offset += length
        
        # Sleep between requests
        time.sleep(sleep_time)
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\n✓ Total records fetched: {len(combined_df):,}")
        return combined_df
    else:
        print("No data fetched.")
        return pd.DataFrame()

In [16]:
eia_plant_inventory = fetch_all_eia_pages(fetch_eia_plant_inventory)
print(f"\nPlant Inventory Data Shape: {eia_plant_inventory.shape}")
print(eia_plant_inventory.head())

Starting pagination for fetch_eia_plant_inventory...
  ✓ Fetched 5,000 records (total so far: 5,000)
  ✓ Fetched 5,000 records (total so far: 10,000)
  ✓ Fetched 5,000 records (total so far: 15,000)
  ✓ Fetched 5,000 records (total so far: 20,000)
  ✓ Fetched 5,000 records (total so far: 25,000)
  ✓ Fetched 5,000 records (total so far: 30,000)
  ✓ Fetched 5,000 records (total so far: 35,000)
  ✓ Fetched 5,000 records (total so far: 40,000)
  ✓ Fetched 5,000 records (total so far: 45,000)
  ✓ Fetched 5,000 records (total so far: 50,000)
  ✓ Fetched 5,000 records (total so far: 55,000)
  ✓ Fetched 5,000 records (total so far: 60,000)
  ✓ Fetched 5,000 records (total so far: 65,000)
  ✓ Fetched 5,000 records (total so far: 70,000)
  ✓ Fetched 5,000 records (total so far: 75,000)
  ✓ Fetched 2,784 records (total so far: 77,784)
Received 2784 records (less than 5000). Pagination complete.

✓ Total records fetched: 77,784

Plant Inventory Data Shape: (77784, 29)
    period stateid stateName 

In [17]:
plant_fuel_mix = fetch_all_eia_pages(fetch_facility_fuel)
print(f"\nFacility Fuel Data Shape: {plant_fuel_mix.shape}")
print(plant_fuel_mix.head())

Starting pagination for fetch_facility_fuel...
  ✓ Fetched 5,000 records (total so far: 5,000)
  ✓ Fetched 5,000 records (total so far: 10,000)
  ✓ Fetched 5,000 records (total so far: 15,000)
  ✓ Fetched 5,000 records (total so far: 20,000)
  ✓ Fetched 5,000 records (total so far: 25,000)
  ✓ Fetched 5,000 records (total so far: 30,000)
  ✓ Fetched 5,000 records (total so far: 35,000)
  ✓ Fetched 5,000 records (total so far: 40,000)
  ✓ Fetched 5,000 records (total so far: 45,000)
  ✓ Fetched 5,000 records (total so far: 50,000)
  ✓ Fetched 5,000 records (total so far: 55,000)
  ✓ Fetched 5,000 records (total so far: 60,000)
  ✓ Fetched 5,000 records (total so far: 65,000)
  ✓ Fetched 5,000 records (total so far: 70,000)
  ✓ Fetched 5,000 records (total so far: 75,000)
  ✓ Fetched 5,000 records (total so far: 80,000)
  ✓ Fetched 5,000 records (total so far: 85,000)
  ✓ Fetched 5,000 records (total so far: 90,000)
  ✓ Fetched 2,260 records (total so far: 92,260)
Received 2260 records (

In [18]:
eia_plant_inventory[['nameplate-capacity-mw', 'net-summer-capacity-mw', 'net-winter-capacity-mw']] = eia_plant_inventory[['nameplate-capacity-mw', 'net-summer-capacity-mw', 'net-winter-capacity-mw']].apply(pd.to_numeric, errors='coerce')
eia_plant_inventory = eia_plant_inventory.groupby(['plantid', 'plantName']).agg({
    'entityid': 'first', 'entityName': 'first',
    'balancing-authority-name': 'first',
    'county': 'first', 'latitude': 'first', 'longitude': 'first',
    'nameplate-capacity-mw': 'sum', 'net-summer-capacity-mw': 'sum', 'net-winter-capacity-mw': 'sum'}).reset_index()
print(f"\nPlant Inventory Data Shape: {eia_plant_inventory.shape}")
print(eia_plant_inventory.head())


Plant Inventory Data Shape: (971, 11)
  plantid                        plantName entityid  \
0   10072  Engineered Carbons Borger Cogen    55727   
1   10154                     Power Island    54769   
2   10167                 Seadrift Coke LP    54705   
3   10184            Central Utility Plant    62903   
4   10243              Ticona Polymers Inc     3432   

                  entityName                     balancing-authority-name  \
0   Orion Engineered Carbons                         Southwest Power Pool   
1              INEOS USA LLC  Electric Reliability Council of Texas, Inc.   
2          Seadrift Coke L P  Electric Reliability Council of Texas, Inc.   
3  Silicon Hills Campus, LLC  Electric Reliability Council of Texas, Inc.   
4        Ticona Polymers Inc  Electric Reliability Council of Texas, Inc.   

       county   latitude    longitude  nameplate-capacity-mw  \
0  Hutchinson   35.66704  -101.432004                  740.0   
1    Brazoria    29.2293     -95.1952  

Check for Missing Values, Duplicate Rows, etc...

In [19]:
print(plant_fuel_mix[~plant_fuel_mix['plantName'].isin(eia_plant_inventory['plantName'])]['plantName'].unique())
print(plant_fuel_mix[~plant_fuel_mix['plantCode'].isin(eia_plant_inventory['plantid'])]['plantCode'].unique())

['Snyder Wind Farm' 'NET Power La Porte Station'
 'TX Dallas 7750 Dynasty Drive' 'Ray Ranch Solar']
['56602' '60910' '66516' '67837']


In [20]:
print(plant_fuel_mix.duplicated().sum())
print(eia_plant_inventory.duplicated().sum())

0
0


In [21]:
plant_fuel_mix = plant_fuel_mix.drop_duplicates()
plant_fuel_mix = plant_fuel_mix[~plant_fuel_mix['plantCode'].isin(['56602', '60910', '66516', '67837'])]

cols_to_convert_f930 = ['generation', 'consumption-for-eg-btu', 'average-heat-content']
plant_fuel_mix[cols_to_convert_f930] = plant_fuel_mix[cols_to_convert_f930].apply(pd.to_numeric, errors='coerce').fillna(0)

In [22]:
eia_plant_inventory.to_csv("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/processed/eia_plant_inventory.csv", index=False)
plant_fuel_mix.to_csv("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/processed/plant_fuel_mix.csv", index=False)

Combine Forms EIA-860M: Monthly Inventory of Operable Electric Generators, and EIA-923: Electric Power Operations (Annual and Monthly) into one dataset. 
<br>Retrieve coordinates from EIA-860 API pull, merge with monthly fuel mix report by electricity generator from EIA-923, and convert latitude and longitude columns into geometry column for spatial analysis.

In [23]:
eia_data = pd.merge(plant_fuel_mix, 
                    eia_plant_inventory[['plantid', 'plantName', 'balancing-authority-name', 'entityid','entityName', 'county', 'latitude', 'longitude', 'nameplate-capacity-mw', 'net-summer-capacity-mw', 'net-winter-capacity-mw']], 
                    left_on=['plantCode', 'plantName'], right_on=['plantid', 'plantName'], 
                    how='left', suffixes=('_fuel', '_inventory'))
eia_data = eia_data.merge(counties, on='county', how='left')
eia_data_gdf = gpd.GeoDataFrame(eia_data, geometry=gpd.points_from_xy(eia_data['longitude'], eia_data['latitude']), crs='EPSG:4326')

In [24]:
eia_data_gdf.head()

,period,plantCode,plantName,fuel2002,fuelTypeDescription,state,stateDescription,primeMover,average-heat-content,average-heat-content-units,...,entityName,county,latitude,longitude,nameplate-capacity-mw,net-summer-capacity-mw,net-winter-capacity-mw,Solar Region,Wind Region,geometry
0,2022-07,9,Copper,ALL,Total,TX,Texas,ALL,0.000,None,...,El Paso Electric Co,El Paso,31.7569,-106.375,3215.3,2331.0,2405.0,FarWest,West,POINT (-106.375 31.7569)
1,2022-07,9,Copper,DFO,Distillate Fuel Oil,TX,Texas,ALL,0.000,MMBtu per barrels,...,El Paso Electric Co,El Paso,31.7569,-106.375,3215.3,2331.0,2405.0,FarWest,West,POINT (-106.375 31.7569)
2,2022-07,9,Copper,DFO,Distillate Fuel Oil,TX,Texas,GT,0.000,MMBtu per barrels,...,El Paso Electric Co,El Paso,31.7569,-106.375,3215.3,2331.0,2405.0,FarWest,West,POINT (-106.375 31.7569)
3,2022-07,9,Copper,NG,Natural Gas,TX,Texas,ALL,1.022,MMBtu per Mcf,...,El Paso Electric Co,El Paso,31.7569,-106.375,3215.3,2331.0,2405.0,FarWest,West,POINT (-106.375 31.7569)
4,2022-07,9,Copper,NG,Natural Gas,TX,Texas,GT,1.022,MMBtu per Mcf,...,El Paso Electric Co,El Paso,31.7569,-106.375,3215.3,2331.0,2405.0,FarWest,West,POINT (-106.375 31.7569)


In [25]:
eia_data_gdf.to_csv("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/processed/eia_data_gdf.csv")

Create Tidy Monthly Electricity Generation Table (Totals and Percentages by Fuel Type) Using Aggregation Methods and Pivot Operations

In [26]:
drop_cols = ['average-heat-content-units', 'consumption-for-eg-btu-units', 'generation-units', 'latitude', 'longitude']  # unit columns to drop from eia_data
monthly_plant_gen = eia_data_gdf.drop(columns=drop_cols)
monthly_plant_gen = monthly_plant_gen[(monthly_plant_gen['fuelTypeDescription'] != 'Total') & (~monthly_plant_gen['fuel2002'].isin(['MWH', 'PUR']))]  # Exclude electricity purchased by power plants from other providers

In [27]:
monthly_plant_gen.to_csv("/home/shalini/projects/academic-research/uiuc/tx-datacenters-geospatial-analysis/data/processed/monthly_plant_gen.csv")

In [28]:
plant_info_cols = ['period', 'plantid', 'plantName', 'balancing-authority-name', 'entityid', 'entityName', 'county', 'geometry', 'Solar Region', 'Wind Region', 'nameplate-capacity-mw','net-summer-capacity-mw', 'net-winter-capacity-mw']

# Total absolute generation by fuel type (MWh)
total_monthly_egen_by_fueltype = monthly_plant_gen.pivot_table(
    index=plant_info_cols,
    columns='fuelTypeDescription',
    values='generation',
    aggfunc='sum',
    fill_value=0).reset_index()

# Add suffix to fuel type generation columns
total_monthly_egen_by_fueltype.rename(columns={col: f"{col}_gen_total" for col in total_monthly_egen_by_fueltype.columns if col not in plant_info_cols}, inplace=True)

fuel_cols = [col for col in total_monthly_egen_by_fueltype.columns if col not in plant_info_cols]

# Compute total generation across fuels grouped by plant_info_cols
total_monthly_egen_by_fueltype['total_generation'] = total_monthly_egen_by_fueltype[fuel_cols].sum(axis=1)

# Drop total_generation column if value is all 0 or NA
if total_monthly_egen_by_fueltype['total_generation'].isna().all() or (total_monthly_egen_by_fueltype['total_generation'] == 0).all():
    total_monthly_egen_by_fueltype.drop(columns='total_generation', inplace=True)
else:
    # Add percentage columns for each fuel type in 'fuelTypeDescription'
    for col in fuel_cols:
        base_name = col.replace('_gen_total', '')  # remove '_gen_total' to avoid suffix duplication on new pct cols
        total_monthly_egen_by_fueltype[f'{base_name}_gen_pct'] = (total_monthly_egen_by_fueltype[col] / total_monthly_egen_by_fueltype['total_generation']) * 100

total_monthly_egen_by_fueltype.sort_values(by='Natural Gas_gen_pct', ascending= False).head(25)

fuelTypeDescription,period,plantid,plantName,balancing-authority-name,entityid,entityName,county,geometry,Solar Region,Wind Region,...,Nuclear_gen_pct,Other_gen_pct,Other Gases_gen_pct,Petroleum Coke_gen_pct,Residual Fuel Oil_gen_pct,Solar_gen_pct,Waste Oil and Other Oils_gen_pct,Wind_gen_pct,Wood Waste Solids_gen_pct,other renewables_gen_pct
12094,2023-12,4195,Powerlane Plant,"Electric Reliability Council of Texas, Inc.",7634,City of Greenville - (TX),Hunt,POINT (-96.1264 33.1707),FarEast,North,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5616,2023-03,4195,Powerlane Plant,"Electric Reliability Council of Texas, Inc.",7634,City of Greenville - (TX),Hunt,POINT (-96.1264 33.1707),FarEast,North,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15069,2024-04,3482,Jones,Southwest Power Pool,17718,Southwestern Public Service Co,Lubbock,POINT (-101.7392 33.5239),NorthWest,Panhandle,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11836,2023-11,64777,WAL456,"Electric Reliability Council of Texas, Inc.",64315,"Walmart Stores Texas, LLC",Cameron,POINT (-97.51944 25.97606),SouthEast,Coastal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11837,2023-11,64778,WAL461,"Electric Reliability Council of Texas, Inc.",64315,"Walmart Stores Texas, LLC",Maverick,POINT (-100.48272 28.70203),SouthEast,South,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11838,2023-11,64791,WAL462,"Electric Reliability Council of Texas, Inc.",64315,"Walmart Stores Texas, LLC",Brazoria,POINT (-95.23078 29.42176),FarEast,Coastal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11839,2023-11,64792,WAL504,"Electric Reliability Council of Texas, Inc.",64315,"Walmart Stores Texas, LLC",Galveston,POINT (-94.83294 29.26514),FarEast,Coastal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11840,2023-11,64793,WAL522,"Electric Reliability Council of Texas, Inc.",64315,"Walmart Stores Texas, LLC",Harris,POINT (-95.06497 29.89446),FarEast,South,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11841,2023-11,64794,WAL529,"Electric Reliability Council of Texas, Inc.",64315,"Walmart Stores Texas, LLC",Galveston,POINT (-95.00045 29.3723),FarEast,Coastal,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11842,2023-11,64795,WAL536,"Electric Reliability Council of Texas, Inc.",64315,"Walmart Stores Texas, LLC",Jones,POINT (-99.69802 32.48078),CenterWest,West,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Aggregate Total Monthly Electricity Generation by Fuel Type Data Across Counties, Solar Regions, and Wind Regions

In [29]:
#list cols in total_monthly_egen_by_fueltype
print(total_monthly_egen_by_fueltype.columns.tolist())

['period', 'plantid', 'plantName', 'balancing-authority-name', 'entityid', 'entityName', 'county', 'geometry', 'Solar Region', 'Wind Region', 'nameplate-capacity-mw', 'net-summer-capacity-mw', 'net-winter-capacity-mw', 'Coal_gen_total', 'Distillate Fuel Oil_gen_total', 'Hydroelectric Conventional_gen_total', 'Municiapl Landfill Gas_gen_total', 'Natural Gas_gen_total', 'Nuclear_gen_total', 'Other_gen_total', 'Other Gases_gen_total', 'Petroleum Coke_gen_total', 'Residual Fuel Oil_gen_total', 'Solar_gen_total', 'Waste Oil and Other Oils_gen_total', 'Wind_gen_total', 'Wood Waste Solids_gen_total', 'other renewables_gen_total', 'total_generation', 'Coal_gen_pct', 'Distillate Fuel Oil_gen_pct', 'Hydroelectric Conventional_gen_pct', 'Municiapl Landfill Gas_gen_pct', 'Natural Gas_gen_pct', 'Nuclear_gen_pct', 'Other_gen_pct', 'Other Gases_gen_pct', 'Petroleum Coke_gen_pct', 'Residual Fuel Oil_gen_pct', 'Solar_gen_pct', 'Waste Oil and Other Oils_gen_pct', 'Wind_gen_pct', 'Wood Waste Solids_gen_p

In [30]:
monthly_agg_county_fuel_mix = total_monthly_egen_by_fueltype.groupby(['county', 'period']).agg({
    # For generation totals, sum across all months/plants
    'Coal_gen_total': 'sum',
    'Distillate Fuel Oil_gen_total': 'sum',
    'Hydroelectric Conventional_gen_total': 'sum',
    'Municiapl Landfill Gas_gen_total': 'sum',
    'Natural Gas_gen_total': 'sum',
    'Nuclear_gen_total': 'sum',
    'Other_gen_total': 'sum',
    'Other Gases_gen_total': 'sum',
    'Petroleum Coke_gen_total': 'sum',
    'Residual Fuel Oil_gen_total': 'sum',
    'Solar_gen_total': 'sum',
    'Waste Oil and Other Oils_gen_total': 'sum',
    'Wind_gen_total': 'sum',
    'Wood Waste Solids_gen_total': 'sum',
    'other renewables_gen_total': 'sum',
    'total_generation': 'sum',
    
    # For percentages, take the mean (since they're already percentages)
    'Coal_gen_pct': 'mean',
    'Distillate Fuel Oil_gen_pct': 'mean',
    'Hydroelectric Conventional_gen_pct': 'mean',
    'Municiapl Landfill Gas_gen_pct': 'mean',
    'Natural Gas_gen_pct': 'mean',
    'Nuclear_gen_pct': 'mean',
    'Other_gen_pct': 'mean',
    'Other Gases_gen_pct': 'mean',
    'Petroleum Coke_gen_pct': 'mean',
    'Residual Fuel Oil_gen_pct': 'mean',
    'Solar_gen_pct': 'mean',
    'Waste Oil and Other Oils_gen_pct': 'mean',
    'Wind_gen_pct': 'mean',
    'Wood Waste Solids_gen_pct': 'mean',
    'other renewables_gen_pct': 'mean',
    
    # Keep first value for regions (should be same within county)
    'Solar Region': 'first',
    'Wind Region': 'first',
    
    # Keep plant capacity info (sum across all plants in county)
    'nameplate-capacity-mw': 'sum',
    'net-summer-capacity-mw': 'sum',
    'net-winter-capacity-mw': 'sum'
}).reset_index()


In [31]:
monthly_agg_county_fuel_mix = counties_gdf.merge(monthly_agg_county_fuel_mix, on=['county', 'Solar Region', 'Wind Region'], how='left')
monthly_agg_county_fuel_mix.head()

,county,Solar Region,Wind Region,geometry,period,Coal_gen_total,Distillate Fuel Oil_gen_total,Hydroelectric Conventional_gen_total,Municiapl Landfill Gas_gen_total,Natural Gas_gen_total,...,Petroleum Coke_gen_pct,Residual Fuel Oil_gen_pct,Solar_gen_pct,Waste Oil and Other Oils_gen_pct,Wind_gen_pct,Wood Waste Solids_gen_pct,other renewables_gen_pct,nameplate-capacity-mw,net-summer-capacity-mw,net-winter-capacity-mw
0,Ochiltree,NorthWest,Panhandle,"POLYGON ((-100.93606 36.4996, -100.91851 36.49...",2022-07,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.0,0.0,0.0,9311.1,9311.1,9311.1
1,Ochiltree,NorthWest,Panhandle,"POLYGON ((-100.93606 36.4996, -100.91851 36.49...",2022-08,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.0,0.0,0.0,9311.1,9311.1,9311.1
2,Ochiltree,NorthWest,Panhandle,"POLYGON ((-100.93606 36.4996, -100.91851 36.49...",2022-09,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.0,0.0,0.0,9311.1,9311.1,9311.1
3,Ochiltree,NorthWest,Panhandle,"POLYGON ((-100.93606 36.4996, -100.91851 36.49...",2022-10,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.0,0.0,0.0,9311.1,9311.1,9311.1
4,Ochiltree,NorthWest,Panhandle,"POLYGON ((-100.93606 36.4996, -100.91851 36.49...",2022-11,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,100.0,0.0,0.0,9311.1,9311.1,9311.1
